In [1]:
# Importing packages
%matplotlib inline
import os
import yolov5
import torch
import cv2
from IPython.display import Image
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import yaml
import random
from pathlib import Path
import tensorflow as tf
import keras
from numpy import asarray
import shutil

# Adding the paths of the whole directory to the system path
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))


# Masking GPU from the system
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

c:\GIS\Projektai\VASA_siuksles\vasa-open-closed-containers\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Loading the configuration file 
with open(os.path.join(os.getcwd(), '..', 'configuration.yml'), 'r') as stream:
    _conf = yaml.safe_load(stream)

In [12]:
def load_models():
    """
    Loads and returns the bbox and the clf models
    """
    # Infering the current file path 
    current_path = os.getcwd()
    
    # Reading the configuration file 
    with open(os.path.join(current_path, '..', 'configuration.yml'), 'r') as stream:
        _conf = yaml.safe_load(stream)
    
    model_bbox_relative = _conf['PREDICTION_MODEL']['model']
    model_bbox_path = os.path.join(current_path, '..', model_bbox_relative)

    # Loading the model to memory
    model_bbox = torch.hub.load('ultralytics/yolov5', 'custom', path=model_bbox_path, force_reload=True)

    # Setting the threshold
    model_bbox.conf = _conf['PREDICTION_MODEL']['threshold']

    # Return the models
    return model_bbox

In [13]:
model_bbox = load_models()

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\rokas.petravicius/.cache\torch\hub\master.zip
YOLOv5  2023-3-8 Python-3.9.13 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 367 layers, 46113663 parameters, 0 gradients
Adding AutoShape... 


In [5]:
def predict(
    image_path: str, 
    model_bbox: yolov5, 
    ) -> list:
    """
    The function crops image and classifies to litter/not litter.

    Arguments
    image_path: str
        path to raw image
    model_bbox: str
        model that predicts the bounding box for a container
    model_tidy: str
        model that classifies whether a container is tidy or not
    conf: dict
        configuration file

    Outputs:
    A list of dictionaries with the following structure: 
    [
        {
            'bbox': [x1, y1, x2, y2],
            'bbox_score': float,
            'tidy_score': float,
            'tidy': bool
        },
        ...
    ]
    """
    
    # Extracting the bounding boxes coordinates
    bboxes = model_bbox(image_path).pandas().xyxy[0]

    # Create list from tensors
    prediction_list = []
    
    # Iterate over all bbox    
    for _, bbox in bboxes.iterrows():
        # Creating an empty dict entry 
        box_dict = {}

        # Get integer values of bbox (ONLY WORKS IF IMAGE HAS ONLY ONE BBOX)
        x1 = int(bbox['xmin'])
        y1 = int(bbox['ymin'])
        x2 = int(bbox['xmax'])
        y2 = int(bbox['ymax'])     

        # Adding the bbox coords to the dict
        box_dict['bbox'] = [x1, y1, x2, y2]

        # Adding the confidence of the bbox 
        box_dict['bbox_score'] = bbox['confidence']
        box_dict['class_name'] = bbox['name']



        # Adding the dict to the list
        prediction_list.append(box_dict)

    return prediction_list

In [6]:
# Creating the prediction directory 
prediction_dir = 'predictions'
if not os.path.exists(prediction_dir):
    os.mkdir(prediction_dir)

# Clearing the contents of the directory 
for file in os.listdir(prediction_dir):
    shutil.rmtree(os.path.join(prediction_dir, file))

In [15]:
# Listing the images from the images/ dir 
image = r'C:\GIS\Projektai\VASA_siuksles\vasa-open-closed-containers\images\DJI_0355_W_0.jpg'

# Iterating over the images 
# Creating a directory in the predictions dir 
image_name = os.path.split(image)[-1].split('.')[0]
image_dir = os.path.join(prediction_dir, image_name)
if not os.path.exists(image_dir):
    os.mkdir(image_dir)

# Reading image with OpenCV
img = cv2.imread(image)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Creating the list with predictions 
predictions = predict(image, model_bbox=model_bbox)
print(f"Number of predictions: {len(predictions)}")

if len(predictions) > 0:
    # Creating first plot for whole image
    fig = plt.figure(figsize=(16, 16))
    # Get x, y coordinates of bbox from predictions list
    i=0
    for pred in predictions:
        x1 = pred['bbox'][0]
        y1 = pred['bbox'][1]
        x2 = pred['bbox'][2]
        y2 = pred['bbox'][3]
        # Drawing rectangle of bbox into the image
        img = cv2.rectangle(img, (x1, y1), (x2,y2), color = (255, 0, 0), thickness=2)
        cv2.putText(img, f"{round(pred['bbox_score']*100)}%", (x1+90, y1-30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 2)
        cv2.putText(img, f"{pred['class_name']}", (x1+90, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 2)
        i+=1
    # Ploting image
    plt.imshow(img)
    plt.axis("off")
    plt.savefig(os.path.join(image_dir, 'prediction_bbox.png'))
    plt.close()

    # # Creating second plot for cropped images
    # fig = plt.figure(figsize=(15, 15))
    # plt.rcParams.update({'text.color': "red",})
    # i=0

Number of predictions: 2
